In [1]:
%pip install gym-super-mario-bros
%pip install -r requirements.txt 
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install 'stable-baselines4[extra]'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
from gym.wrappers import GrayScaleObservation
import os

In [10]:
from nes_py.wrappers import JoypadSpace
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from stable_baselines3.common.env_util import make_vec_env
env = gym.make('SuperMarioBros-v0') 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grey scale
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [12]:
done = True
env.reset()
for step in range(500):
    action = env.action_space.sample()
    obs, reward, done, info = env.step([action])
    # env.render()
    if done:
       state = env.reset()

In [13]:
import torch
torch.cuda.is_available()

True

In [34]:
def evaluate_model(model, env, num_episodes=5):
    """
    Evaluate the performance of the model on a given environment.

    Parameters:
    - model: The trained model to evaluate.
    - env: The environment on which to evaluate the model.
    - num_episodes: Number of episodes to run for the evaluation.

    Returns:
    - avg_reward: The average reward obtained over the evaluation episodes.
    """
    total_rewards = 0.0
    for ep in range(num_episodes):
        obs = env.reset()
        done = False
        rewards = 0.0
        while not done:
            action = model.predict(obs)[0]
            obs, reward, done, info = env.step(action)
            # print(reward)
            rewards += reward[0]
            if done:
                break
        total_rewards += rewards
        print(f"Episode {ep+1}: {rewards=:.2f}")
    avg_reward = total_rewards / num_episodes
    return avg_reward


In [18]:
model = PPO.load("ppo_mario")

In [ ]:
evaluate_model(model, env, num_episodes=5)

In [18]:
import torch

In [3]:
time = 1_000_000
lr = 0.0000365514567388135
batch_size = 2048
n_epochs = 10
clip_range = 0.279291059130489
n_steps = 1024
gamma = 0.9951330516817
ent_coef = 0.00290321362474043

seed = 43
callback = CheckpointCallback(save_freq=100_000, save_path='./gym/')
hyperparam = f'{time=}, {lr=}, {ent_coef=}, {batch_size=}, {n_epochs=}, {clip_range=}, {n_steps=}, {gamma=}'
model = PPO("CnnPolicy", env, verbose=1, learning_rate=lr, n_steps=n_steps, ent_coef=ent_coef, batch_size=batch_size, n_epochs=n_epochs, clip_range=clip_range, gamma=gamma, tensorboard_log="logs", seed=seed)
model.learn(total_timesteps=time, callback=callback, tb_log_name=hyperparam)

model.save("ppo_mario_{hyperparam}")

Using cuda device
Wrapping the env in a VecTransposeImage.
Logging to logs/time=2000000, lr=3.65514567388135e-05, ent_coef=0.00290321362474043, batch_size=2048, n_epochs=10, clip_range=0.279291059130489, n_steps=1024, gamma=0.9951330516817_1


/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


-----------------------------
| time/              |      |
|    fps             | 273  |
|    iterations      | 1    |
|    time_elapsed    | 14   |
|    total_timesteps | 4096 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 47          |
|    iterations           | 2           |
|    time_elapsed         | 172         |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.001818073 |
|    clip_fraction        | 0           |
|    clip_range           | 0.279       |
|    entropy_loss         | -2.48       |
|    explained_variance   | -0.00148    |
|    learning_rate        | 3.66e-05    |
|    loss                 | 7.86        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00261    |
|    value_loss           | 21.6        |
-----------------------------------------
----------------------------------

In [15]:
model=PPO.load('gym/rl_model_1100000_steps')

In [21]:
evaluate_model(model, env, num_episodes=5)

Episode 1: rewards=1242.00
Episode 2: rewards=1519.00
Episode 3: rewards=1420.00
Episode 4: rewards=1136.00
Episode 5: rewards=1118.00


1287.0

In [22]:
os.chdir('gym')

In [27]:
os.listdir()

['rl_model_100000_steps.zip',
 'best_model.zip',
 'rl_model_1400000_steps.zip',
 'rl_model_900000_steps.zip',
 'rl_model_1200000_steps.zip',
 'rl_model_500000_steps.zip',
 'rl_model_1500000_steps.zip',
 'rl_model_400000_steps.zip',
 'rl_model_700000_steps.zip',
 'rl_model_200000_steps.zip',
 'rl_model_1600000_steps.zip',
 'rl_model_1700000_steps.zip',
 'rl_model_1100000_steps.zip',
 'rl_model_1800000_steps.zip',
 'rl_model_800000_steps.zip',
 'rl_model_1000000_steps.zip',
 'rl_model_900000_steps (2).zip',
 'rl_model_1300000_steps.zip',
 'rl_model_600000_steps.zip',
 'rl_model_300000_steps.zip']

In [30]:
mp = {}

In [31]:
for file in os.listdir():
    if file.endswith('.zip') and file != 'rl_model_900000_steps (2).zip':
        model = PPO.load(file)
        print(file)
        # evaluate_model(model, env, num_episodes=5)
        mp[file] = evaluate_model(model, env, num_episodes=5)
       

rl_model_100000_steps.zip
Episode 1: rewards=755.00
Episode 2: rewards=632.00
Episode 3: rewards=755.00
Episode 4: rewards=504.00
Episode 5: rewards=632.00
best_model.zip
Episode 1: rewards=752.00
Episode 2: rewards=710.00
Episode 3: rewards=431.00
Episode 4: rewards=503.00
Episode 5: rewards=504.00
rl_model_1400000_steps.zip
Episode 1: rewards=1215.00
Episode 2: rewards=961.00
Episode 3: rewards=1408.00
Episode 4: rewards=1373.00
Episode 5: rewards=1022.00
rl_model_900000_steps.zip
Episode 1: rewards=1367.00
Episode 2: rewards=1275.00
Episode 3: rewards=1445.00
Episode 4: rewards=1368.00
Episode 5: rewards=1022.00
rl_model_1200000_steps.zip
Episode 1: rewards=1048.00
Episode 2: rewards=1064.00
Episode 3: rewards=1147.00
Episode 4: rewards=1003.00
Episode 5: rewards=1365.00
rl_model_500000_steps.zip
Episode 1: rewards=1422.00
Episode 2: rewards=961.00
Episode 3: rewards=1298.00
Episode 4: rewards=1026.00
Episode 5: rewards=1387.00
rl_model_1500000_steps.zip
Episode 1: rewards=1290.00
E

In [32]:
mp

{'rl_model_100000_steps.zip': 655.6,
 'best_model.zip': 580.0,
 'rl_model_1400000_steps.zip': 1195.8,
 'rl_model_900000_steps.zip': 1295.4,
 'rl_model_1200000_steps.zip': 1125.4,
 'rl_model_500000_steps.zip': 1218.8,
 'rl_model_1500000_steps.zip': 1212.8,
 'rl_model_400000_steps.zip': 680.0,
 'rl_model_700000_steps.zip': 1131.8,
 'rl_model_200000_steps.zip': 711.4,
 'rl_model_1600000_steps.zip': 852.2,
 'rl_model_1700000_steps.zip': 1115.0,
 'rl_model_1100000_steps.zip': 1266.4,
 'rl_model_1800000_steps.zip': 1145.0,
 'rl_model_800000_steps.zip': 1080.8,
 'rl_model_1000000_steps.zip': 886.8,
 'rl_model_1300000_steps.zip': 876.4,
 'rl_model_600000_steps.zip': 1188.6,
 'rl_model_300000_steps.zip': 902.0}

In [33]:
from stable_baselines3.common.evaluation import evaluate_policy

In [36]:
model = PPO.load('rl_model_900000_steps')

In [40]:
evaluate_model(model, env, num_episodes=5)

Episode 1: rewards=1367.00
Episode 2: rewards=1275.00
Episode 3: rewards=1445.00
Episode 4: rewards=1368.00
Episode 5: rewards=1022.00


1295.4